# Exercise 1.1

### Packages and Files

In [1]:
import cv2
import math
import numpy as np

### Class Declaration for trackable object

##### Each identified object will be mapped to a trackable class object where information such as:

- center position of current object at frame
- object id tagged to each object
- the count to check if the object has already been marked (for those moving towards town)

##### will be stored and processed

In [2]:
class TrackedObjects:
    def __init__(self, objectID, centroid):

        # store the object ID, then initialize a list of centroids
        self.objectID = objectID

        # using the current centroid
        self.centroid = centroid

        # initialize a boolean used to indicate if the object has been counted
        self.counted = False

### Functions

#### Get background Model

##### By using a random sampling size of 60 frames to generate a background image

In [3]:
# Required for background subtraction later on
def get_background(file_path):
    cap = cv2.VideoCapture(file_path)
    # 60 frames were selected as a form of median for calculation later on
    frame_indices = cap.get(cv2.CAP_PROP_FRAME_COUNT) * np.random.uniform(size=60)
    # storing the frames in array
    frames = []
    for idx in frame_indices:
        # set frame id to read that particular frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        #read frame
        ret, frame = cap.read()
        #add frame to array
        frames.append(frame)
    # calculate the median
    median_frame = np.median(frames, axis=0).astype(np.uint8)
    # return median function
    return median_frame

#### Get centroids and draw rectangles

##### For every frame that is access, the objects will be identified on ONLY the main street (excluding humans and bicycles), rectangles will be drawn over the vehicles while each vehicle's center position will be appended into a higher global array based on the euclidean distance as defined at a later section of this notebook

In [4]:
def get_centroids_draw_rectangles(all_contours):
    for contour in all_contours:
        # continue through the loop if contour area is less than 500...
        # ... helps in removing noise detection
        if cv2.contourArea(contour) < 500:
            continue
        # get the x-min, y-min, width, and height coordinates from the contours
        (x, y, w, h) = cv2.boundingRect(contour)

        # Cropping box detection to only bottom half of the screen
        if y >= 250:
            # Ensure detection is not human or bicycle
            if w > 62 and h > 84:
                # draw the bounding boxes
                cv2.rectangle(orig_frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

                #Get centroid of each box
                x,y = get_centroid(x,y,w,h)
                current_centroid = [x,y]
                current_frame_centroids.append(current_centroid)

#### Check object if new or existing

##### This function runs checks on every new frame of objects against the previous frame objects to determine if the are either new item where it will be added into the frame or existing item (determine by the eulidean distance of previous frame centriod to current frame centriod), where the deviation amount will not be more than 40 regardless of magnitude and the new value will be updated onto the existing object by on nearest distance

In [5]:
def check_object_addition_or_updates(current_frame,previous_frame,counted):
    for centroid in current_frame:
        #created boolean to check if object is new
        not_found = True
        #Check if centroid is already from the previous DataFrame
        for item in previous_frame:

            #Check if euclidean < 20
            if 0 <= abs(math.dist(centroid, item.centroid)) < 60:
                #Assign new centroid to existing centroid
                item.centroid = centroid
                #print("object" + str(item.objectID) + "updated!" + " Values: " + str(item.centroid[0]) + " " +  str(item.centroid[1]))
                not_found = False

        if not_found:
            #Add new centroid to object list
            counted = counted + 1
            #print("object" + str(counted_cars) + "added!" + " Values: " + str(current_centroid[0]) + " " +  str(current_centroid[1]))
            objects.append(TrackedObjects(counted_cars,centroid))

#### Check object if out of frame

##### This function ensures that when an object leaves the specific area that we are analysing, it will be remove from the array to prevent cluttering of unwanted data causing unnecessary errors from surfacing.

In [6]:
def check_out_of_frame(previous_frame,current_frame):
    for item in previous_frame:

        #created boolean to check if object is out of frame
        not_found = True
        for centroid in current_frame:
            #Check if euclidean < 20
            if 0 <= abs(math.dist(centroid, item.centroid)) < 60:
                not_found = False

        if not_found:
            #Remove out of frame objects
            #print("object" + str(item.objectID) + "removed!" + " Values: " + str(item.centroid[0]) + " " + str(item.centroid[1]))
            objects.remove(item)

#### Check object if travelling to downtown (Task 2 Requirements)

##### This function checks for vehicles moving towards town. This particular implementation has been done through the crossing of space technique where vehicles will be marked as cross the moment it passes across a specific marked out area. In this case, we demarcated the area (570 to 640 for x-axis and 360 to 480 for y-axis) as that is the optimal spot to obtain traffic towards downtown.

In [7]:
def check_downtown_marker_crossing(current_frame):
    global total_car
    for item in current_frame:
        # Check if vehicle is moving towards town
        # Downtown marker coordinates
        # x-axis = 480
        # y-axis = 340 - 460
        if (570 < item.centroid[0] <= 640) and (360 < item.centroid[1] <= 480):
            if item.counted:
                pass
            else:
                print("Car detected!")
                item.counted = True
                total_car = total_car + 1
                print("Total Car: ", str(total_car))

### Algorithm for loading video file into frames for further processing

In [8]:
# Open Video
cap = cv2.VideoCapture('Traffic_Laramie_1.mp4')

# get the background model
background = get_background('Traffic_Laramie_1.mp4')
# convert the background model to grayscale format
background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
fps = cap.get(cv2.CAP_PROP_FPS)
total_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
total_duration = total_frame_count/fps
frame_count = 0
# declare the no of consecutive frames to analyse, the smaller the higher the accuracy of each rectangle being drawn on each frame's object(s)
consecutive_frame = 4
counted_cars = 0
total_car = 0
objects: list = []

#### This is the algorithm to calculated the center of each contour's 4 corners after applying the rect function

In [9]:
def get_centroid(x,y,w,h):
    return x + (w / 2), y + (h / 2)

### Processing motion detection

In [10]:
# Loop to start processing each current and previous frame
while cap.isOpened():
    ret, frame = cap.read()
    # Check if frame available
    if ret == True:
        frame_count += 1
        #Duplicate frame to ensure original colored frame stays
        orig_frame = frame.copy()
        # Convert the frame to grayscale first
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        #Check if frame is at the very start of the processing queue
        if frame_count % consecutive_frame == 0 or frame_count == 1:
            frame_diff_list = []
        # find the difference between current frame and base frame
        frame_diff = cv2.absdiff(gray, background)
        # thresholding to convert the frame to binary
        ret, thres = cv2.threshold(frame_diff, 50, 255, cv2.THRESH_BINARY)
        # dilate the frame a bit to get some more white area...
        # ... makes the detection of contours a bit easier
        dilate_frame = cv2.dilate(thres, None, iterations=2)
        # append the final result into the `frame_diff_list`
        frame_diff_list.append(dilate_frame)

        if len(frame_diff_list) == consecutive_frame:
            # add all the frames in the `frame_diff_list`
            sum_frames = sum(frame_diff_list)
            # find the contours around the white segmented areas
            contours, hierarchy = cv2.findContours(sum_frames, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            # draw the contours, not strictly necessary
            for i, cnt in enumerate(contours):
                cv2.drawContours(frame, contours, i, (0, 0, 255), 3)

            #create new array for temporary checking of out of frame objects
            current_frame_centroids = []

            # Get all relevant box [x,y] values and append to current_frame_centroids
            get_centroids_draw_rectangles(contours)

            # Check all the existing objects and new objects
            check_object_addition_or_updates(current_frame_centroids,objects,counted_cars)

            # Check all the out of frame objects
            check_out_of_frame(objects, current_frame_centroids)

            # Check if line crosses
            check_downtown_marker_crossing(objects)

            cv2.imshow('Detected Objects', orig_frame)
            if cv2.waitKey(100) & 0xFF == ord('q'):
                break
    else:
        break
total_car_per_minute = round((1/(total_duration/60)) * total_car)
print("Cars per minute: ", str(total_car_per_minute))
cap.release()
cv2.destroyAllWindows()

Car detected!
Total Car:  1
Car detected!
Total Car:  2
Car detected!
Total Car:  3
Car detected!
Total Car:  4
Cars per minute:  2
